In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.trainer.trainer_classifier import Trainer_classifier
from src.trainer.trainer_VAE import Trainer_VAE
from src.models.classifiers import *
from src.trainer.model_class import Model_class
from src.trainer.loss_class import Loss_class

from src.models.autoencoders import *
from src.models.joinedModel import *

import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from sklearn.cluster import KMeans

In [11]:
df = pd.read_csv('../data/df_to_enc.csv')

In [12]:
df = df.drop(columns=['Machine failure']).to_numpy()

In [15]:
df = df.astype(float)

In [18]:
df.dtype

dtype('float64')

In [41]:
from sklearn.preprocessing import StandardScaler

In [43]:
df_scale = StandardScaler().fit(df).transform(df)

In [44]:
k_dist = KMeans(n_clusters=2, random_state=42).fit_transform(df_scale)

In [46]:
kmeans = KMeans(n_clusters=2, random_state=42).fit(df_scale)

In [47]:
cluster_centers = kmeans.cluster_centers_

In [48]:
distances = torch.cdist(torch.tensor(df), torch.tensor(cluster_centers), p=2)
softmax_weights = nn.functional.softmax(-distances, dim=1)

In [49]:
softmax_weights

tensor([[0.8109, 0.1891],
        [0.8076, 0.1924],
        [0.8047, 0.1953],
        ...,
        [0.8191, 0.1809],
        [0.8055, 0.1945],
        [0.8130, 0.1870]], dtype=torch.float64)

In [37]:
distances

tensor([[ 77960.2671,  23311.3358],
        [119420.9015,  20481.7999],
        [ 95182.4962,  13306.9804],
        ...,
        [ 46730.0272,  52989.0146],
        [118787.5724,  19664.9872],
        [ 90361.8278,  10022.9936]], dtype=torch.float64)